In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
df = pd.read_csv(
    r'E:\MSE\HomeWork\Ky 2\DAM501.8\Test & Project\Project\Project\new_retail_data.csv')


In [ ]:
df.columns


In [ ]:
df.Product_Category.nunique()


In [ ]:
print(df[['Year', 'Month', 'Customer_Segment']])


In [ ]:
columns_to_drop = [
    'Transaction_ID', 'Name', 'Email', 'Phone', 'Address', 'Age', 'State', 'Country', 'Income',
    'Zipcode', 'Time', 'Amount', 'Total_Purchases', 'Feedback', 'Order_Status', 'products', 'Shipping_Method'
]

df.drop(columns=columns_to_drop, inplace=True)


In [ ]:
df.columns


In [ ]:
df


In [ ]:
df.isnull().sum()


In [ ]:
df.dropna(inplace=True)


In [ ]:
df


In [ ]:
df.duplicated().sum()


In [ ]:
df.drop_duplicates(inplace=True)


In [ ]:
# Fix mixed date formats
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


In [ ]:
# Standardize categorical values (remove whitespace, consistent casing)
categorical_cols = ['City', 'Gender', 'Customer_Segment', 'Month',
                    'Product_Category', 'Product_Brand', 'Product_Type', 'Payment_Method']
df[categorical_cols] = df[categorical_cols].apply(
    lambda x: x.str.strip().str.title())


In [ ]:
df.head()


In [ ]:
# Check rows with invalid or missing dates
invalid_dates = df[df['Date'].isna()]
print(invalid_dates[['Date']])


In [ ]:
# Since Month and Year columns provide similar information, Date is dropped
df.drop(columns=['Date'], inplace=True)


In [ ]:
df.sort_values(by='Customer_ID', ascending=True, inplace=True)


In [ ]:
df.head()


In [ ]:
# it seems to be of different store so with same customer id there is difference in gender, and customer_segment.
# As this is not real analysis I want to make this simple .
# I want to treat one customer is as a single person , so For each Customer_ID, fill in Gender and Customer_Segment
# using the most frequent value (mode) within that customer’s records.

# Calculate mode for each customer
gender_mode = df.groupby('Customer_ID')['Gender'].agg(
    lambda x: x.mode().iloc[0])
segment_mode = df.groupby('Customer_ID')['Customer_Segment'].agg(
    lambda x: x.mode().iloc[0])

# Map the mode values back to the original DataFrame
df['Gender'] = df['Customer_ID'].map(gender_mode)
df['Customer_Segment'] = df['Customer_ID'].map(segment_mode)


In [ ]:
df.head()


# Visualization


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 5))
sns.boxplot(data=df, x='Gender', y='Total_Amount')
plt.title('Spending Distribution by Gender')
plt.show()


Both Male and Female have similar Spending Patterns.


In [ ]:
df


# 1. Top Product Categories by Total Sales


In [ ]:
plt.figure(figsize=(12, 6))
category_sales = df.groupby('Product_Category')[
    'Total_Amount'].sum().sort_values(ascending=False)
sns.barplot(x=category_sales.index, y=category_sales.values, palette='Blues_d')
plt.title('Total Sales by Product Category')
plt.xlabel('Product Category')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# 2. Customer Count by Gender


In [ ]:
# Get mode of gender per unique customer
customer_gender = df.groupby('Customer_ID')['Gender'].agg(
    lambda x: x.mode().iat[0]).reset_index()

# Plot the count of unique customers by gender
plt.figure(figsize=(6, 4))
sns.countplot(x='Gender', data=customer_gender, palette='pastel')
plt.title('Unique Customer Count by Gender')
plt.xlabel('Gender')
plt.ylabel('Number of Unique Customers')
plt.tight_layout()
plt.show()


# 3. Total Sales by Payment Method


In [ ]:
plt.figure(figsize=(10, 5))
payment_sales = df.groupby('Payment_Method')[
    'Total_Amount'].sum().sort_values(ascending=False)
sns.barplot(x=payment_sales.index, y=payment_sales.values, palette='viridis')
plt.title('Total Sales by Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.show()


# 4. Distribution of Ratings


In [ ]:
plt.figure(figsize=(8, 4))
sns.histplot(df['Ratings'], bins=5, kde=True, color='skyblue')
plt.title('Distribution of Customer Ratings')
plt.xlabel('Ratings')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()


# 5. Monthly Sales Trend


In [ ]:
# Ensure month order
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Group by Year and Month
monthly_sales = df.groupby(['Year', 'Month'])[
    'Total_Amount'].sum().reset_index()

# Make sure Month is a categorical type for proper sorting
monthly_sales['Month'] = pd.Categorical(
    monthly_sales['Month'], categories=month_order, ordered=True)
monthly_sales = monthly_sales.sort_values(['Year', 'Month'])

# Plot
plt.figure(figsize=(10, 6))
sns.lineplot(data=monthly_sales, x='Month', y='Total_Amount',
             hue='Year', marker='o', palette='tab10')
plt.title('Monthly Sales Trend (2023 vs 2024)')
plt.ylabel('Total Sales Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# 6. Sales by Customer Segment


In [ ]:
plt.figure(figsize=(8, 5))
segment_sales = df.groupby('Customer_Segment')[
    'Total_Amount'].sum().sort_values(ascending=False)
sns.barplot(x=segment_sales.index, y=segment_sales.values, palette='Set2')
plt.title('Sales by Customer Segment')
plt.xlabel('Customer Segment')
plt.ylabel('Total Sales')
plt.tight_layout()
plt.show()


# 7. Top Cities by Total Sales


In [ ]:
plt.figure(figsize=(12, 6))
top_cities = df.groupby('City')['Total_Amount'].sum(
).sort_values(ascending=False).head(10)
sns.barplot(x=top_cities.index, y=top_cities.values, palette='magma')
plt.title('Top 10 Cities by Total Sales')
plt.xlabel('City')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# 8. Customer Spending by Product Brand


In [ ]:
# Group by Product_Brand and sum the Total_Amount
brand_spending = df.groupby('Product_Brand')[
    'Total_Amount'].sum().sort_values(ascending=False)

# Plot
plt.figure(figsize=(12, 6))
sns.barplot(x=brand_spending.index, y=brand_spending.values, palette='viridis')
plt.title('Customer Spending by Product Brand')
plt.xlabel('Product Brand')
plt.ylabel('Total Spending Amount')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


In [ ]:
df


In [ ]:
df.to_csv('customer_transaction_category_data.csv')
